In [1]:
import azureml

from azureml.core import Run
from azureml.core import Workspace

from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.environment import Environment

from azureml.core.experiment import Experiment

from azureml.core.image import Image

from azureml.core.model import InferenceConfig
from azureml.core.model import Model

from azureml.core.run import Run

from azureml.core.webservice import AciWebservice, Webservice

StatementMeta(spark5, 0, 3, Finished, Available)

In [2]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = b"

#Provide values for the existing Resource Group 
resource_group09-1"

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_n6tqjlmi"
workspace_region = "East US"

model_name = "batt-cycles-7"

description = 'Forecast'
cpu_cores = 1
memory_gb = 1
tags = {'name':'scoring'}

conda_packages = ['numpy', 'pandas', 'scikit-learn', 'py-xgboost<=0.80']
pip_packages = ['azureml-train-automl==1.0.60', 'inference-schema']
conda_file = "dependencies.yml"

env_name = "myenv"
scoring_file = "scoring_service.py"

se-aciservice-3uka6z6tqjlmi"

StatementMeta(spark5, 0, 4, Finished, Available)

In [3]:
# Values from `Setup Service Principal Authentication` in the Batch_Scoring notebook - do that first, then get those values and paste them here
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47" # Use "tenantId" value
service_principal_id = "df87bb49-ae74-40f8-baf8-e7e4474d4ad7" # Use "clientId" value
service_principal_password = "A~GWfa7VAvqJp.9UT5aCGPyiP6LAVVgZkh" # Use "clientSecret" value


StatementMeta(spark5, 0, 5, Submitted, Available)

In [ ]:
# ws = Workspace.create(
#     name = workspace_name,
#     subscription_id = subscription_id,
#     resource_group = resource_group,
#     location = workspace_region,
#     exist_ok = True)

In [4]:
from azureml.core.authentication import ServicePrincipalAuthentication

sp = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=service_principal_id,
    service_principal_password=service_principal_password)

StatementMeta(spark5, 0, 6, Finished, Available)

In [5]:
from azureml.core import Workspace

ws = Workspace.get(
    name=workspace_name, 
    auth=sp,
    subscription_id=subscription_id)

StatementMeta(spark5, 0, 7, Finished, Available)

In [6]:
ws.get_details()

StatementMeta(spark5, 0, 8, Finished, Available)

{'id': '/subscriptions/220fc532-6091-423c-8ba0-66c2397d591b/resourceGroups/pz-iot-20200709-1/providers/Microsoft.MachineLearningServices/workspaces/Cosmos-DB-IoT-ML-3uka6z6tqjlmi', 'name': 'Cosmos-DB-IoT-ML-3uka6z6tqjlmi', 'location': 'eastus', 'type': 'Microsoft.MachineLearningServices/workspaces', 'sku': 'Basic', 'workspaceid': 'c8caadf6-18ce-499d-8553-e7c4c4278cc2', 'description': '', 'friendlyName': 'Cosmos-DB-IoT-ML-3uka6z6tqjlmi', 'creationTime': '2020-08-14T17:11:44.9449442+00:00', 'keyVault': '/subscriptions/220fc532-6091-423c-8ba0-66c2397d591b/resourcegroups/pz-iot-20200709-1/providers/microsoft.keyvault/vaults/iot-vault-3uka6z6tqjlmi', 'applicationInsights': '/subscriptions/220fc532-6091-423c-8ba0-66c2397d591b/resourcegroups/pz-iot-20200709-1/providers/microsoft.insights/components/cosmos-db-iot-insights-3uka6z6tqjlmi', 'identityPrincipalId': 'a3f287df-5c11-4c7f-8ed9-f783cf48be57', 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47', 'identityType': 'SystemAssigned', '

In [19]:
scoring_service = """
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

# from inference_schema.schema_decorators import input_schema, output_schema
# from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
# from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

input_sample = pd.DataFrame({'battery_Age_Days': 120, 'daily_Trip_Duration': 590})
### input_sample = pd.DataFrame(data=[{"Date":"2013-01-01T00:00:00.000Z","Battery_ID":0,"Battery_Age_Days":0,"Daily_Trip_Duration":67.8456075842}])

def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = Model.get_model_path(model_name = 'batt-cycles-7')
    model = joblib.load(model_path)

@input_schema('data', PandasParameterType(input_sample))
def run(data):
    try:
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

    return json.dumps({"result": result.tolist()})
"""

with open(scoring_file, "w") as file:
  file.write(scoring_service)

StatementMeta(spark5, 0, 21, Finished, Available)

1174

In [20]:
registeredModel = Model(workspace = ws, name = model_name)

StatementMeta(spark5, 0, 22, Finished, Available)

In [21]:
registeredModel

StatementMeta(spark5, 0, 23, Finished, Available)

Model(workspace=Workspace.create(name='Cosmos-DB-IoT-ML-3uka6z6tqjlmi', subscription_id='220fc532-6091-423c-8ba0-66c2397d591b', resource_group='pz-iot-20200709-1'), name=batt-cycles-7, id=batt-cycles-7:1, version=1, tags={}, properties={})

In [22]:
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = cpu_cores, 
    memory_gb = memory_gb, 
    tags = tags, 
    description = description)

StatementMeta(spark5, 0, 24, Finished, Available)

In [23]:
conda_env = CondaDependencies.create(conda_packages = conda_packages, pip_packages = pip_packages)

with open(conda_file, "w") as f:
    f.write(conda_env.serialize_to_string())

StatementMeta(spark5, 0, 25, Finished, Available)

596

In [ ]:
import os

files = [f for f in os.listdir('.') if os.path.isfile(f)]

for f in files:
    print(f)

In [24]:
myenv = Environment.from_conda_specification(name = env_name, file_path = conda_file)

myenv.register(workspace = ws)

StatementMeta(spark5, 0, 26, Finished, Available)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
              

In [25]:
inference_config = InferenceConfig(entry_script = scoring_file, environment = myenv)

StatementMeta(spark5, 0, 27, Finished, Available)

In [26]:
inference_config

StatementMeta(spark5, 0, 28, Finished, Available)

InferenceConfig(entry_script=scoring_service.py, runtime=None, conda_file=None, extra_docker_file_steps=None, source_directory=None, enable_gpu=None, base_image=None, base_image_registry=<azureml.core.container_registry.ContainerRegistry object at 0x7fe5c4cdb9e8>)

In [27]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

StatementMeta(spark5, 0, 29, Finished, Available)

In [28]:
deployment_config

StatementMeta(spark5, 0, 30, Finished, Available)

In [29]:
service = Model.deploy(ws, service_name, [registeredModel], inference_config, deployment_config, overwrite = True)

StatementMeta(spark5, 0, 31, Finished, Available)

In [30]:
service.wait_for_deployment(show_output = True)

print(service.state)

StatementMeta(spark5, 0, 32, Finished, Available)

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cb4cd878-2dbb-43a0-b794-744789b408fb
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: aciservice-3uka6z6tqjlmi. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image c8caadf618ce499d8553e7c4c4278cc2.azurecr.io/azureml/azureml_3d65de966e82d21990ac57cf225e7267 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scori

In [ ]:
print(service.get_logs())

StatementMeta(, , , Waiting, )